In [1]:
import numpy as np
import torch as t

In [2]:
import torch_geometric
import torch_geometric.nn as pygnn

In [3]:
A = t.tensor([[0,1,1,0],[1,0,1,1],[1,1,0,0],[0,1,0,0]])

In [4]:
COO = A.nonzero().t().contiguous()
X = t.tensor([1,0,3,20], dtype=t.float)
X = X.view(X.size()[0], 1)
data = torch_geometric.data.Data(X,edge_index=COO,y=t.tensor([0,1,0,0], dtype=t.float))
data

Data(x=[4, 1], edge_index=[2, 8], y=[4])

In [5]:
X.size()

torch.Size([4, 1])

In [6]:
COO

tensor([[0, 0, 1, 1, 1, 2, 2, 3],
        [1, 2, 0, 2, 3, 0, 1, 1]])

In [7]:
t.tensor([[1,2,5],[5,6,7],[2,6,3]], dtype=t.float) @ t.eye(3)

tensor([[1., 2., 5.],
        [5., 6., 7.],
        [2., 6., 3.]])

In [8]:
class Net(t.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = pygnn.GCNConv(1, 1) 
        

    def forward(self, data:torch_geometric.data.Data):
        x = data.x
        x = self.conv1(x, data.edge_index)
        print(">>>", x)
        x = t.nn.functional.relu(x)
        return x.softmax(dim=0)
        

In [9]:
model = Net()
model.train()
optimizor = t.optim.SGD(model.parameters(), lr=0.5)

In [10]:
for epoch in range(100):
    optimizor.zero_grad()
    y_hat = model(data)
    print(y_hat.view(-1), data.y, y_hat.softmax(0))
    loss = t.nn.functional.binary_cross_entropy(y_hat.view(-1), data.y)
    loss.backward()
    print(loss.item())
    optimizor.step()
    
    

>>> tensor([[ 1.8135],
        [11.1878],
        [ 1.8135],
        [13.6009]], grad_fn=<AddBackward0>)
tensor([6.9746e-06, 8.2177e-02, 6.9746e-06, 9.1781e-01],
       grad_fn=<ViewBackward0>) tensor([0., 1., 0., 0.]) tensor([[0.1789],
        [0.1942],
        [0.1789],
        [0.4480]], grad_fn=<SoftmaxBackward0>)
1.249403715133667
>>> tensor([[1.2705],
        [7.8382],
        [1.2705],
        [9.5288]], grad_fn=<AddBackward0>)
tensor([2.1866e-04, 1.5562e-01, 2.1866e-04, 8.4394e-01],
       grad_fn=<ViewBackward0>) tensor([0., 1., 0., 0.]) tensor([[0.1820],
        [0.2127],
        [0.1820],
        [0.4233]], grad_fn=<SoftmaxBackward0>)
0.9295639991760254
>>> tensor([[0.7698],
        [4.7491],
        [0.7698],
        [5.7735]], grad_fn=<AddBackward0>)
tensor([0.0049, 0.2616, 0.0049, 0.7286], grad_fn=<ViewBackward0>) tensor([0., 1., 0., 0.]) tensor([[0.1867],
        [0.2414],
        [0.1867],
        [0.3851]], grad_fn=<SoftmaxBackward0>)
0.6637478470802307
>>> tensor([[0.

In [11]:
from torch_geometric.datasets import Planetoid

In [12]:
dataset = Planetoid('./data', name='Cora')

In [13]:
def toLogits(X):
    print('X is ', X)
    val = X.view(-1).detach().softmax(0)
    val.apply_(lambda a: a > 0.25)
    return val

In [14]:
toLogits(model(data))

>>> tensor([[0.1335],
        [0.8237],
        [0.1335],
        [1.0014]], grad_fn=<AddBackward0>)
X is  tensor([[0.1568],
        [0.3128],
        [0.1568],
        [0.3736]], grad_fn=<SoftmaxBackward0>)


tensor([0., 1., 0., 1.])

In [15]:
toLogits(model(data))

>>> tensor([[0.1335],
        [0.8237],
        [0.1335],
        [1.0014]], grad_fn=<AddBackward0>)
X is  tensor([[0.1568],
        [0.3128],
        [0.1568],
        [0.3736]], grad_fn=<SoftmaxBackward0>)


tensor([0., 1., 0., 1.])

In [26]:
import networkx as nx
import pygraphviz

a = torch_geometric.utils.to_networkx(data, to_undirected=True)
A = nx.nx_agraph.to_agraph(a)
A.layout()  # neato layout
A.draw("file.png")  # write postscript in k5.ps with neato layout
